In [1]:
import pandas as pd
import numpy as np
import joblib
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
# Scaling is generally not needed/helpful for TF-IDF with these models
# from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss, f1_score, make_scorer
from sklearn.exceptions import ConvergenceWarning, NotFittedError
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers, optimizers # Import optimizers
import sys
import gc
import time # For timing

# --- Configuration ---
DATASET_PATH = 'dataset.csv'
TEXT_COLUMN = 'report'
RANDOM_STATE = 42
VAL_SIZE = 0.20 # Validation set size (as fraction of initial non-test data)
TEST_SIZE = 0.20 # Test set size (as fraction of total data)

# --- Hyperparameter Tuning Grids ---
TFIDF_MAX_FEATURES = 5000 # Keep features reasonable

# Grids kept similar, let GridSearch find best within these reasonable ranges
LR_PARAM_GRID = {
    'estimator__C': [0.1, 1, 10],
    'estimator__class_weight': ['balanced', None] # Add option without balancing
}

SVC_PARAM_GRID = {
    'estimator__C': [0.1, 1, 10],
    'estimator__max_iter': [2000, 3000],
    'estimator__class_weight': ['balanced', None]
}

# Increase regularization strength options for Perceptron
PERCEPTRON_PARAM_GRID = {
    'estimator__eta0': [0.001, 0.01],
    'estimator__alpha': [0.01, 0.1, 0.5], # Increased alpha options
    'estimator__penalty': ['l2', 'elasticnet'],
    'estimator__max_iter': [1000, 2000],
    'estimator__class_weight': ['balanced', None]
}

# --- DNN Parameters (Adjusted for less overfitting) ---
DNN_HIDDEN_UNITS_1 = 128 # Reduced slightly
DNN_HIDDEN_UNITS_2 = 64  # Reduced slightly
DNN_DROPOUT_RATE_1 = 0.6 # Increased
DNN_DROPOUT_RATE_2 = 0.5 # Increased
DNN_L2_REG = 0.005     # Increased L2 regularization
DNN_EPOCHS = 50
DNN_BATCH_SIZE = 64     # Slightly larger batch size
DNN_LEARNING_RATE = 0.0005 # Slightly reduced learning rate
DNN_EARLY_STOPPING_PATIENCE = 10 # Increased patience

# --- Online Perceptron Parameters (Adjusted for less overfitting) ---
ONLINE_PERCEPTRON_ETA = 0.005 # Adjusted
ONLINE_PERCEPTRON_ALPHA = 0.01 # Stronger regularization
ONLINE_PERCEPTRON_EPOCHS = 5

# --- Evaluation Parameters ---
PRECISION_K_VALUES = [3, 5] # Calculate Precision@3 and Precision@5

# --- Suppress Warnings ---
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.multiclass')
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.metrics._classification')

# --- 1. Load and Analyze Data ---
print("Loading dataset...")
try:
    df = pd.read_csv(DATASET_PATH)
except FileNotFoundError:
    print(f"Error: Dataset file not found at {DATASET_PATH}")
    sys.exit(1)

label_columns = [col for col in df.columns if col.startswith('type_')]
if not label_columns:
    print("Error: No label columns found starting with 'type_'. Check CSV header.")
    sys.exit(1)
print(f"Identified Label Columns: {label_columns}")

print("\nDataset Info:")
df.info()

print("\nMissing Values:")
print(df.isnull().sum())
if df[TEXT_COLUMN].isnull().any():
    print(f"Warning: Missing values found in '{TEXT_COLUMN}'. Filling with empty strings.")
    df[TEXT_COLUMN].fillna('', inplace=True)

print("\nLabel Distribution:")
label_dist = df[label_columns].sum()
print(label_dist)
if (label_dist == 0).any():
    print("\nWarning: The following label columns have zero positive instances:")
    zero_labels = label_dist[label_dist == 0].index.tolist()
    print(zero_labels)
    print("Metrics for these labels might be zero or undefined.")
else:
    zero_labels = []


X = df[TEXT_COLUMN]
y = df[label_columns]
num_labels_total = y.shape[1]

# --- 2. Preprocessing ---
print(f"\nPreprocessing text data with TF-IDF (max_features={TFIDF_MAX_FEATURES})...")
vectorizer = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)
print(f"TF-IDF Matrix Shape: {X_tfidf.shape}")

# --- 3. Train/Validation/Test Split ---
print(f"\nSplitting data into Train ({1-TEST_SIZE:.0%}), Test ({TEST_SIZE:.0%})...")
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_tfidf, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y.sum(axis=1)>0
)
print(f"Splitting Train+Val data into Train ({1-VAL_SIZE:.0%}) and Validation ({VAL_SIZE:.0%})...")
val_split_adjusted = VAL_SIZE / (1.0 - TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=val_split_adjusted, random_state=RANDOM_STATE, stratify=y_train_val.sum(axis=1)>0
)

y_train_np = y_train.values
y_val_np = y_val.values
y_test_np = y_test.values

print(f"Final Shapes -> Train: {X_train.shape} / {y_train_np.shape}")
print(f"Final Shapes -> Validation: {X_val.shape} / {y_val_np.shape}")
print(f"Final Shapes -> Test: {X_test.shape} / {y_test_np.shape}")


# --- 4. Evaluation Functions ---

def precision_at_k(y_true, y_scores, k):
    """Calculates Precision@k for multi-label data."""
    if y_scores is None:
        return np.nan
    if not isinstance(y_true, np.ndarray): y_true = np.array(y_true)
    if not isinstance(y_scores, np.ndarray): y_scores = np.array(y_scores)
    if y_true.shape[0] == 0: return 0.0 # Handle empty input

    top_k_preds_indices = np.argsort(y_scores, axis=1)[:, -k:]
    precision_sum = 0
    num_samples = y_true.shape[0]

    for i in range(num_samples):
        true_labels_indices = np.where(y_true[i] == 1)[0]
        if len(true_labels_indices) == 0:
             sample_precision = 0.0 # Or 1.0 if k=0? Let's stick to 0.0
        else:
            top_k_set = set(top_k_preds_indices[i])
            true_set = set(true_labels_indices)
            intersection = len(top_k_set.intersection(true_set))
            sample_precision = intersection / k
        precision_sum += sample_precision
    return precision_sum / num_samples

def evaluate_multilabel_full(y_true, y_pred_binary, y_pred_scores=None):
    """Calculates comprehensive multi-label metrics."""
    metrics = {}
    set_name = "Unknown" # Placeholder for dataset type
    if np.array_equal(y_true, y_test_np): set_name = "Test"
    elif np.array_equal(y_true, y_val_np): set_name = "Validation"
    elif np.array_equal(y_true, y_train_np): set_name = "Training"

    if y_true.shape[0] == 0: # Handle empty slices
        print(f"  Warning: Empty y_true provided for evaluation ({set_name} set). Skipping metrics.")
        metrics = {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan}
        for k in PRECISION_K_VALUES: metrics[f"Precision@{k}"] = np.nan
        return metrics

    if y_true.shape != y_pred_binary.shape:
        print(f"Error: Shape mismatch during evaluation ({set_name} set). y_true: {y_true.shape}, y_pred: {y_pred_binary.shape}")
        metrics = {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan}
        for k in PRECISION_K_VALUES: metrics[f"Precision@{k}"] = np.nan
        return metrics

    try:
        metrics["Hamming Loss"] = hamming_loss(y_true, y_pred_binary)
        metrics["Micro-F1"] = f1_score(y_true, y_pred_binary, average='micro', zero_division=0)
        active_labels = [i for i, label in enumerate(label_columns) if label not in zero_labels]
        if not active_labels:
            print(f"Warning ({set_name} set): No active labels with positive instances found for Macro-F1 calculation.")
            metrics["Macro-F1"] = 0.0
        else:
            metrics["Macro-F1"] = f1_score(y_true, y_pred_binary, average='macro', zero_division=0, labels=active_labels)

        # Calculate Precision@k if scores are available
        for k in PRECISION_K_VALUES:
            metrics[f"Precision@{k}"] = precision_at_k(y_true, y_pred_scores, k) if y_pred_scores is not None else np.nan

        # Format Precision@k part of the string only if values are not NaN
        pk_str = ", ".join([f"P@{k}={metrics[f'Precision@{k}']:.4f}" for k in PRECISION_K_VALUES if f"Precision@{k}" in metrics and not np.isnan(metrics[f'Precision@{k}'])])
        print(f"  Metrics ({set_name}): Hamming Loss={metrics['Hamming Loss']:.4f}, "
              f"Micro-F1={metrics['Micro-F1']:.4f}, Macro-F1={metrics['Macro-F1']:.4f}" +
              (f", {pk_str}" if pk_str else "")) # Add comma only if pk_str is not empty
        return metrics
    except Exception as e:
         print(f"Error during evaluation ({set_name} set): {e}")
         metrics = {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan}
         for k in PRECISION_K_VALUES: metrics[f"Precision@{k}"] = np.nan
         return metrics

# --- 5. Model Training, Tuning, and Evaluation ---

results_train = {}
results_test = {}
results_val = {}
best_params = {}

micro_f1_scorer = make_scorer(f1_score, average='micro', zero_division=0)

# --- Logistic Regression (Tuning and Evaluation) ---
model_name = "Logistic Regression"
print(f"\n--- Tuning and Training {model_name} (OvR) ---")
start_time = time.time()
base_lr = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE, max_iter=2000) # Increased max_iter
ovr_lr = OneVsRestClassifier(base_lr)
grid_lr = GridSearchCV(ovr_lr, LR_PARAM_GRID, cv=3, scoring=micro_f1_scorer, n_jobs=-1, verbose=1)
grid_lr.fit(X_train, y_train_np)
best_lr = grid_lr.best_estimator_
best_params[model_name] = grid_lr.best_params_
print(f"Best Params: {best_params[model_name]}")
print(f"Tuning Time: {time.time() - start_time:.2f} seconds")

print(f"\n--- Evaluating Best {model_name} ---")
y_pred_lr_test_scores = best_lr.decision_function(X_test) if hasattr(best_lr, 'decision_function') else None
y_pred_lr_test = best_lr.predict(X_test)
results_test[model_name] = evaluate_multilabel_full(y_test_np, y_pred_lr_test, y_pred_lr_test_scores)
y_pred_lr_val_scores = best_lr.decision_function(X_val) if hasattr(best_lr, 'decision_function') else None
y_pred_lr_val = best_lr.predict(X_val)
results_val[model_name] = evaluate_multilabel_full(y_val_np, y_pred_lr_val, y_pred_lr_val_scores)
y_pred_lr_train_scores = best_lr.decision_function(X_train) if hasattr(best_lr, 'decision_function') else None
y_pred_lr_train = best_lr.predict(X_train)
results_train[model_name] = evaluate_multilabel_full(y_train_np, y_pred_lr_train, y_pred_lr_train_scores)


# --- SVM (LinearSVC - Tuning and Evaluation) ---
model_name = "SVM (LinearSVC)"
print(f"\n--- Tuning and Training {model_name} (OvR) ---")
start_time = time.time()
base_svm = LinearSVC(random_state=RANDOM_STATE, dual='auto') # dual='auto' safer
ovr_svm = OneVsRestClassifier(base_svm)
grid_svm = GridSearchCV(ovr_svm, SVC_PARAM_GRID, cv=3, scoring=micro_f1_scorer, n_jobs=-1, verbose=1)
grid_svm.fit(X_train, y_train_np)
best_svm = grid_svm.best_estimator_
best_params[model_name] = grid_svm.best_params_
print(f"Best Params: {best_params[model_name]}")
print(f"Tuning Time: {time.time() - start_time:.2f} seconds")

print(f"\n--- Evaluating Best {model_name} ---")
y_pred_svm_test_scores = best_svm.decision_function(X_test)
y_pred_svm_test = best_svm.predict(X_test)
results_test[model_name] = evaluate_multilabel_full(y_test_np, y_pred_svm_test, y_pred_svm_test_scores)
y_pred_svm_val_scores = best_svm.decision_function(X_val)
y_pred_svm_val = best_svm.predict(X_val)
results_val[model_name] = evaluate_multilabel_full(y_val_np, y_pred_svm_val, y_pred_svm_val_scores)
y_pred_svm_train_scores = best_svm.decision_function(X_train)
y_pred_svm_train = best_svm.predict(X_train)
results_train[model_name] = evaluate_multilabel_full(y_train_np, y_pred_svm_train, y_pred_svm_train_scores)


# --- Perceptron (Batch - Tuning and Evaluation) ---
model_name = "Perceptron (Batch)"
print(f"\n--- Tuning and Training {model_name} (OvR) ---")
start_time = time.time()
base_perceptron = Perceptron(random_state=RANDOM_STATE, tol=1e-3,
                             early_stopping=True, validation_fraction=0.1, n_iter_no_change=5)
ovr_perceptron = OneVsRestClassifier(base_perceptron)
# Added stronger regularization options to grid
grid_perceptron = GridSearchCV(ovr_perceptron, PERCEPTRON_PARAM_GRID, cv=3, scoring=micro_f1_scorer, n_jobs=-1, verbose=1)
grid_perceptron.fit(X_train, y_train_np)
best_perceptron_batch = grid_perceptron.best_estimator_
best_params[model_name] = grid_perceptron.best_params_
print(f"Best Params: {best_params[model_name]}")
print(f"Tuning Time: {time.time() - start_time:.2f} seconds")

print(f"\n--- Evaluating Best {model_name} ---")
# Skipping P@k for Perceptron
y_pred_perceptron_batch_test = best_perceptron_batch.predict(X_test)
results_test[model_name] = evaluate_multilabel_full(y_test_np, y_pred_perceptron_batch_test, None)
y_pred_perceptron_batch_val = best_perceptron_batch.predict(X_val)
results_val[model_name] = evaluate_multilabel_full(y_val_np, y_pred_perceptron_batch_val, None)
y_pred_perceptron_batch_train = best_perceptron_batch.predict(X_train)
results_train[model_name] = evaluate_multilabel_full(y_train_np, y_pred_perceptron_batch_train, None)


# --- Challenge: Perceptron (Online Learning) ---
model_name = "Perceptron (Online)"
print(f"\n--- Training {model_name} (OvR - No Tuning) ---")
start_time = time.time()
perceptron_model_online = OneVsRestClassifier(
    Perceptron(random_state=RANDOM_STATE, eta0=ONLINE_PERCEPTRON_ETA, warm_start=True, max_iter=1,
               penalty='l2', alpha=ONLINE_PERCEPTRON_ALPHA, class_weight='balanced') # Kept balanced for online
)

n_epochs = ONLINE_PERCEPTRON_EPOCHS
classes = np.array([0, 1])
all_classes = [classes] * y_train_np.shape[1]
online_model_initialized = False
online_model_fitted = False

# Initial partial_fit call
initial_batch_size = min(5, X_train.shape[0])
try:
    print(f"Initializing {model_name} structure using {initial_batch_size} samples...")
    perceptron_model_online.label_binarizer_ = perceptron_model_online.label_binarizer_.fit(y_train_np)
    perceptron_model_online.partial_fit(X_train[0:initial_batch_size], y_train_np[0:initial_batch_size], classes=all_classes)
    print("Initialization complete.")
    online_model_initialized = True
except Exception as e_init:
    print(f"Error during {model_name} initialization: {e_init}. Skipping this model.")
    results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})

if online_model_initialized:
    print(f"Starting online training for {n_epochs} epochs...")
    for epoch in range(n_epochs):
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        y_train_shuffled = y_train_np[indices]
        processed_samples = 0
        for idx in indices:
            sample_X = X_train[idx]
            sample_y = y_train_shuffled[idx:idx+1]
            try:
                 perceptron_model_online.partial_fit(sample_X, sample_y)
                 processed_samples += 1
                 online_model_fitted = True
            except Exception as e_partial:
                 print(f"Warning: Error during partial_fit for sample {idx} in epoch {epoch+1}: {e_partial}")
        print(f"Epoch {epoch+1}/{n_epochs} completed. Processed {processed_samples}/{X_train.shape[0]} samples in this epoch.")

    print(f"Online training finished. Time: {time.time() - start_time:.2f} seconds")
    if online_model_fitted:
        try:
            print(f"\n--- Evaluating {model_name} ---")
            y_pred_perceptron_online_test = perceptron_model_online.predict(X_test)
            results_test[model_name] = evaluate_multilabel_full(y_test_np, y_pred_perceptron_online_test, None)
            y_pred_perceptron_online_val = perceptron_model_online.predict(X_val)
            results_val[model_name] = evaluate_multilabel_full(y_val_np, y_pred_perceptron_online_val, None)
            y_pred_perceptron_online_train = perceptron_model_online.predict(X_train)
            results_train[model_name] = evaluate_multilabel_full(y_train_np, y_pred_perceptron_online_train, None)
        except Exception as e_pred_online:
             print(f"Error predicting or evaluating {model_name}: {e_pred_online}")
             results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
             results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
             results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    else:
        print(f"{model_name} did not successfully fit any data.")
        results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
        results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
        results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
else:
    print(f"Skipping {model_name} evaluation due to initialization failure.")
    results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})


# --- Deep Neural Network (DNN) ---
model_name = "DNN"
print(f"\n--- Training {model_name} Model ---")
start_time = time.time()
dnn_model_multilabel = None
X_train_dense = None
X_val_dense = None
X_test_dense = None

try:
    print("Converting sparse matrices to dense for DNN (this may take significant memory)...")
    X_train_dense = X_train.toarray()
    X_val_dense = X_val.toarray()
    X_test_dense = X_test.toarray()
    print("Conversion to dense complete.")
except MemoryError:
    print("Error: Insufficient memory to convert TF-IDF matrix to dense array for DNN.")
    print(f"Skipping {model_name} training and evaluation.")
    results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
except Exception as e_dense:
    print(f"An unexpected error occurred during dense conversion: {e_dense}")
    print(f"Skipping {model_name} training and evaluation.")
    results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
    results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})

if X_train_dense is not None and X_val_dense is not None and X_test_dense is not None:
    # DNN definition with adjusted regularization
    def build_dnn_multilabel(input_shape, output_shape):
        model = Sequential([
            Input(shape=(input_shape,)),
            Dense(DNN_HIDDEN_UNITS_1, activation='relu', kernel_regularizer=regularizers.l2(DNN_L2_REG)),
            BatchNormalization(),
            Dropout(DNN_DROPOUT_RATE_1),
            Dense(DNN_HIDDEN_UNITS_2, activation='relu', kernel_regularizer=regularizers.l2(DNN_L2_REG)),
            BatchNormalization(),
            Dropout(DNN_DROPOUT_RATE_2),
            Dense(output_shape, activation='sigmoid')
        ])
        model.compile(optimizer=optimizers.Adam(learning_rate=DNN_LEARNING_RATE), # Use optimizers module
                      loss='binary_crossentropy',
                      metrics=['binary_accuracy',
                               tf.keras.metrics.Precision(name='precision'),
                               tf.keras.metrics.Recall(name='recall'),
                               tf.keras.metrics.AUC(name='auc', multi_label=True)])
        return model

    num_labels = y_train_np.shape[1]
    dnn_model_multilabel = build_dnn_multilabel(X_train_dense.shape[1], num_labels)
    dnn_model_multilabel.summary()

    early_stopping_dnn = EarlyStopping(monitor='val_loss', patience=DNN_EARLY_STOPPING_PATIENCE, restore_best_weights=True, verbose=1)

    print("Fitting DNN model...")
    history_dnn = dnn_model_multilabel.fit(X_train_dense, y_train_np,
                                           epochs=DNN_EPOCHS,
                                           batch_size=DNN_BATCH_SIZE,
                                           validation_data=(X_val_dense, y_val_np),
                                           callbacks=[early_stopping_dnn],
                                           verbose=1) # Set verbose=1 to see epoch progress
    print(f"DNN Training Time: {time.time() - start_time:.2f} seconds")

    print(f"\n--- Evaluating {model_name} ---")
    try:
        y_pred_dnn_proba_ml_test = dnn_model_multilabel.predict(X_test_dense)
        y_pred_dnn_ml_test = (y_pred_dnn_proba_ml_test > 0.5).astype(int)
        results_test[model_name] = evaluate_multilabel_full(y_test_np, y_pred_dnn_ml_test, y_pred_dnn_proba_ml_test)

        y_pred_dnn_proba_ml_val = dnn_model_multilabel.predict(X_val_dense)
        y_pred_dnn_ml_val = (y_pred_dnn_proba_ml_val > 0.5).astype(int)
        results_val[model_name] = evaluate_multilabel_full(y_val_np, y_pred_dnn_ml_val, y_pred_dnn_proba_ml_val)

        y_pred_dnn_proba_ml_train = dnn_model_multilabel.predict(X_train_dense)
        y_pred_dnn_ml_train = (y_pred_dnn_proba_ml_train > 0.5).astype(int)
        results_train[model_name] = evaluate_multilabel_full(y_train_np, y_pred_dnn_ml_train, y_pred_dnn_proba_ml_train)
    except Exception as e_eval_dnn:
        print(f"Error during DNN evaluation: {e_eval_dnn}")
        results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
        results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
        results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
else:
     results_test.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
     results_val.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})
     results_train.setdefault(model_name, {"Hamming Loss": np.nan, "Micro-F1": np.nan, "Macro-F1": np.nan})

# --- 7. Final Comparison ---
print("\n" + "="*30 + " FINAL RESULTS " + "="*30)

def format_results_df(results_dict, title):
    print(f"\n--- {title} ---")
    valid_results = {k: v for k, v in results_dict.items() if v and not all(np.isnan(list(v.values())))}
    if valid_results:
        cols = ["Hamming Loss", "Micro-F1", "Macro-F1"] + [f"Precision@{k}" for k in PRECISION_K_VALUES]
        df_results = pd.DataFrame(valid_results).T
        df_results = df_results.reindex(columns=cols) # Ensure all columns are present
        print(df_results.sort_values(by="Micro-F1", ascending=False).to_string(float_format="%.4f"))
        return df_results
    else:
        print(f"No valid results to display for {title}.")
        return None

results_df_test = format_results_df(results_test, "TEST SET RESULTS")
results_df_val = format_results_df(results_val, "VALIDATION SET RESULTS")
results_df_train = format_results_df(results_train, "TRAINING SET RESULTS")


print("\n--- Best Hyperparameters Found (GridSearchCV) ---")
for model, params in best_params.items():
    print(f"{model}: {params}")


print("\n--- Overfitting Check (Comparing Train vs Test Micro-F1) ---")
comparison_data = []
# Use available results dataframes for comparison
train_index = results_df_train.index if results_df_train is not None else []
test_index = results_df_test.index if results_df_test is not None else []
common_models = set(train_index).intersection(set(test_index))

for model_name in sorted(list(common_models)): # Iterate through models present in both train and test results
    train_metrics = results_df_train.loc[model_name]
    test_metrics = results_df_test.loc[model_name]
    train_f1 = train_metrics.get("Micro-F1", np.nan)
    test_f1 = test_metrics.get("Micro-F1", np.nan)
    train_macro = train_metrics.get("Macro-F1", np.nan)
    test_macro = test_metrics.get("Macro-F1", np.nan)

    comparison_data.append({
        "Model": model_name,
        "Train Micro-F1": train_f1,
        "Test Micro-F1": test_f1,
        "Micro Diff": train_f1 - test_f1 if not np.isnan(train_f1) and not np.isnan(test_f1) else np.nan,
        "Train Macro-F1": train_macro,
        "Test Macro-F1": test_macro,
        "Macro Diff": train_macro - test_macro if not np.isnan(train_macro) and not np.isnan(test_macro) else np.nan,
    })

if comparison_data:
     comparison_df = pd.DataFrame(comparison_data)
     # Sort by Test Micro F1 before printing
     print(comparison_df.sort_values(by="Test Micro-F1", ascending=False).to_string(index=False, float_format="%.4f"))
     overfit_threshold = 0.10 # Lowered threshold slightly
     comparison_df['Overfitting (Micro-F1)'] = comparison_df['Micro Diff'] > overfit_threshold
     print("\nPotential Overfitting based on Micro-F1 Difference >", overfit_threshold)
     overfitting_models = comparison_df[comparison_df['Overfitting (Micro-F1)'] == True]
     if not overfitting_models.empty:
        print(overfitting_models[['Model', 'Train Micro-F1', 'Test Micro-F1', 'Micro Diff']].to_string(index=False, float_format="%.4f"))
     else:
        print("No significant overfitting detected based on the threshold.")
else:
     print("No comparison data available.")


# --- Explicitly delete large dense arrays and run garbage collection ---
print("\nCleaning up dense arrays...")
del X_train_dense
del X_val_dense
del X_test_dense
X_train_dense = None
X_val_dense = None
X_test_dense = None
gc.collect()
print("Cleanup complete.")


print("\nPart 2 Processing Complete.")
print("\n--- Saving Part 2 Artifacts ---")
output_dir_p2 = "part2_artifacts"
os.makedirs(output_dir_p2, exist_ok=True)

# Save TF-IDF Vectorizer
joblib.dump(vectorizer, os.path.join(output_dir_p2, "vectorizer_p2.joblib"))
print("TF-IDF Vectorizer saved.")

# Save Tuned Sklearn Models (using the best estimators)
part2_models_to_save = {
    "Logistic Regression": best_lr,
    "SVM (LinearSVC)": best_svm,
    "Perceptron (Batch)": best_perceptron_batch
    # Not saving Online Perceptron as it's less practical for deployment
}
for name, model in part2_models_to_save.items():
     filename = name.lower().replace(" ", "_").replace("(","").replace(")","") + "_p2.joblib"
     joblib.dump(model, os.path.join(output_dir_p2, filename))
     print(f"{name} model saved.")

# Save DNN model (if trained successfully)
if dnn_model_multilabel:
    dnn_model_multilabel.save(os.path.join(output_dir_p2, "dnn_model_p2.keras"))
    print("DNN model saved.")
else:
    print("DNN model was not trained successfully, skipping save.")

# Save label columns list
part2_config = {'label_columns': label_columns}
with open(os.path.join(output_dir_p2, 'part2_config.pkl'), 'wb') as f:
     pickle.dump(part2_config, f)
print("Part 2 config (label columns) saved.")

print("Part 2 Artifact saving complete.")

Loading dataset...
Identified Label Columns: ['type_blocker', 'type_regression', 'type_bug', 'type_documentation', 'type_enhancement', 'type_task', 'type_dependency_upgrade']

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   report                   1386 non-null   object
 1   type_blocker             1386 non-null   int64 
 2   type_regression          1386 non-null   int64 
 3   type_bug                 1386 non-null   int64 
 4   type_documentation       1386 non-null   int64 
 5   type_enhancement         1386 non-null   int64 
 6   type_task                1386 non-null   int64 
 7   type_dependency_upgrade  1386 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 86.8+ KB

Missing Values:
report                     0
type_blocker               0
type_regression            0
type_bug           

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         640,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 649,607 (2.48 MB)

 Trainable params: 649,223 (2.48 MB)

 Non-trainable params: 384 (1.50 KB)

Fitting DNN model...
Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - auc: 0.4591 - binary_accuracy: 0.5101 - loss: 2.5586 - precision: 0.2928 - recall: 0.5006 - val_auc: 0.5238 - val_binary_accuracy: 0.7019 - val_loss: 1.9645 - val_precision: 0.4830 - val_recall: 0.5874
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - auc: 0.5158 - binary_accuracy: 0.5448 - loss: 2.1501 - precision: 0.3235 - recall: 0.5400 - val_auc: 0.5842 - val_binary_accuracy: 0.6973 - val_loss: 1.7596 - val_precision: 0.4731 - val_recall: 0.5063
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - auc: 0.5621 - binary_accuracy: 0.5535 - loss: 1.9441 - precision: 0.3380 - recall: 0.5819 - val_auc: 0.6499 - val_binary_accuracy: 0.7004 - val_loss: 1.6396 - val_precision: 0.4780 - val_recall: 0.5081
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - auc: 0.6040 - binary_accuracy: 0.5795 - loss: 1.7779 - precision: 0.3600 - recall: 0.5996 - val_auc: 0.6788 - val_binary_accuracy: 0.6988 - val_loss: 1.5634 - va